In [249]:
import pandas as pd
import sklearn.feature_extraction.text as tx, sklearn.naive_bayes as nb, sklearn.linear_model as lm, sklearn.ensemble as en, sklearn.metrics as me, sklearn.model_selection as ms
from warnings import filterwarnings as fw
fw('ignore')

все слова с учетом регистра<br>
все в одном регистре<br>
без стоп слов<br>
выделение специфичных для классов<br>
-- -- 
лог регрессия, случайный лес, байес

In [245]:
ord('Н')%3

0

In [276]:
def fitpredict(xtrain,ytrain,xtest,ytest):
    
    lr =  lm.LogisticRegression()
    rfc = en.RandomForestClassifier()
    cnb = nb.ComplementNB()

    lr.fit(xtrain,ytrain)
    rfc.fit(xtrain,ytrain)
    cnb.fit(xtrain,ytrain)

    pred_lr = lr.predict(xtest)
    pred_rfc = rfc.predict(xtest)
    pred_cnb = cnb.predict(xtest)
    
    print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
        'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
        'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))
    
    return lr, rfc, cnb

In [246]:
corpus = pd.read_csv('SMSpam.csv')
corpus['Message'] = corpus['Message'].str.replace(r'[^\w\s]+','')

In [247]:
cnt, cntS, cntH = 0, 0, 0
for i in corpus['Message']:
    cnt += len(i.split(' '))
for i in corpus[corpus['Category']=='spam']['Message']:
    cntS += len(i.split(' '))
for i in corpus[corpus['Category']=='ham']['Message']:
    cntH += len(i.split(' '))
print('Всего строк:', corpus.shape[0], '\nИз них', 
    corpus['Category'].value_counts()['spam'], 'spam и',corpus['Category'].value_counts()['ham'], 'ham', 
    '\nВсего слов: ', cnt, '\nиз них', cntS, 'spam и', cntH, 'ham')

Всего строк: 5572 
Из них 747 spam и 4825 ham 
Всего слов:  87265 
из них 17788 spam и 69477 ham


In [254]:
xtrain, xtest, ytrain, ytest = ms.train_test_split(corpus['Message'],corpus['Category'], test_size=0.2)
print('Обучающая выборка:', xtrain.shape,'\nТестовая выборка', xtest.shape)

Обучающая выборка: (4457,) 
Тестовая выборка (1115,)


-- -- 
Используем все слова с учетом регистра:

In [267]:
cv = tx.CountVectorizer(lowercase=False)
cv.fit(corpus['Message'])
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 11596 токенов


In [279]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97446   0.99687   0.98554       957
        spam    0.97794   0.84177   0.90476       158

    accuracy                        0.97489      1115
   macro avg    0.97620   0.91932   0.94515      1115
weighted avg    0.97496   0.97489   0.97409      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.96960   1.00000   0.98457       957
        spam    1.00000   0.81013   0.89510       158

    accuracy                        0.97309      1115
   macro avg    0.98480   0.90506   0.93984      1115
weighted avg    0.97391   0.97309   0.97189      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99023   0.95298   0.97125       957
        spam    0.76804   0.94304   0.84659       158

    accuracy                        0.95157      1115
   macro avg    0.87913   0.94801   0.90892      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Используем все слова, но в одном регистре:

In [280]:
cv = tx.CountVectorizer()
cv.fit(corpus['Message'])
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 9542 токенов


In [281]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97546   0.99687   0.98605       957
        spam    0.97810   0.84810   0.90847       158

    accuracy                        0.97578      1115
   macro avg    0.97678   0.92248   0.94726      1115
weighted avg    0.97583   0.97578   0.97505      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.96569   1.00000   0.98255       957
        spam    1.00000   0.78481   0.87943       158

    accuracy                        0.96951      1115
   macro avg    0.98285   0.89241   0.93099      1115
weighted avg    0.97055   0.96951   0.96793      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98813   0.95716   0.97240       957
        spam    0.78191   0.93038   0.84971       158

    accuracy                        0.95336      1115
   macro avg    0.88502   0.94377   0.91106      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Найдем стоп-слова и уберем их:

In [297]:
cv = tx.CountVectorizer(max_df=0.01)
cv.fit(corpus['Message'])
stop = cv.stop_words_
print('Всего', len(stop), 'токенов')
#print(stop)

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 204 токенов


In [298]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.95892   1.00000   0.97903       957
        spam    1.00000   0.74051   0.85091       158

    accuracy                        0.96323      1115
   macro avg    0.97946   0.87025   0.91497      1115
weighted avg    0.96474   0.96323   0.96087      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.96468   0.99896   0.98152       957
        spam    0.99194   0.77848   0.87234       158

    accuracy                        0.96771      1115
   macro avg    0.97831   0.88872   0.92693      1115
weighted avg    0.96854   0.96771   0.96605      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99309   0.90073   0.94466       957
        spam    0.61538   0.96203   0.75062       158

    accuracy                        0.90942      1115
   macro avg    0.80424   0.93138   0.84764      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

Используем встроенный список стоп-слов:

In [303]:
cv = tx.CountVectorizer(stop_words = 'english')
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.96862   1.00000   0.98406       957
        spam    1.00000   0.80380   0.89123       158

    accuracy                        0.97220      1115
   macro avg    0.98431   0.90190   0.93764      1115
weighted avg    0.97307   0.97220   0.97091      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.96667   1.00000   0.98305       957
        spam    1.00000   0.79114   0.88339       158

    accuracy                        0.97040      1115
   macro avg    0.98333   0.89557   0.93322      1115
weighted avg    0.97139   0.97040   0.96893      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99330   0.92999   0.96060       957
        spam    0.69406   0.96203   0.80637       158

    accuracy                        0.93453      1115
   macro avg    0.84368   0.94601   0.88349      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Используем только слова с высокой важностью:

In [312]:
cv = tx.CountVectorizer()
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

e = en.RandomForestClassifier()
e.fit(tr_xtrain,ytrain)
e.score(tr_xtest,ytest)

0.9704035874439462

In [313]:
imp = pd.DataFrame({'name' : cv.vocabulary_.keys(), 'importance' : e.feature_importances_}) 
imp = imp.sort_values(by = 'importance', ascending = False)
top100 = imp.head(n=1000)['name'].to_numpy()

cv = tx.CountVectorizer(vocabulary = top100)
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.91715   0.99478   0.95439       957
        spam    0.93506   0.45570   0.61277       158

    accuracy                        0.91839      1115
   macro avg    0.92611   0.72524   0.78358      1115
weighted avg    0.91969   0.91839   0.90598      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.93558   0.98642   0.96033       957
        spam    0.87736   0.58861   0.70455       158

    accuracy                        0.93004      1115
   macro avg    0.90647   0.78751   0.83244      1115
weighted avg    0.92733   0.93004   0.92408      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.96175   0.89342   0.92633       957
        spam    0.54867   0.78481   0.64583       158

    accuracy                        0.87803      1115
   macro avg    0.75521   0.83911   0.78608      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Используем tf-idf векторизатор:

In [314]:
tfidfv = tx.TfidfVectorizer()
tfidfv.fit(corpus['Message'])
tr_xtrain = tfidfv.transform(xtrain)
tr_xtest = tfidfv.transform(xtest)

In [315]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.95319   1.00000   0.97603       957
        spam    1.00000   0.70253   0.82528       158

    accuracy                        0.95785      1115
   macro avg    0.97659   0.85127   0.90066      1115
weighted avg    0.95982   0.95785   0.95467      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.96862   1.00000   0.98406       957
        spam    1.00000   0.80380   0.89123       158

    accuracy                        0.97220      1115
   macro avg    0.98431   0.90190   0.93764      1115
weighted avg    0.97307   0.97220   0.97091      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98620   0.97074   0.97841       957
        spam    0.83815   0.91772   0.87613       158

    accuracy                        0.96323      1115
   macro avg    0.91217   0.94423   0.92727      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())